 To calculate the potential for yearly reduction on CO2 emissions, caused by the taxi cabs roaming without passengers. In your calculation please assume that the taxi cab fleet is changing at the rate of 15% per month (from combustion engine-powered vehicles to electric vehicles). Assume also that the average passenger vehicle emits about 404 grams of CO2 per mile.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from geopy import distance
from global_land_mask import globe
import os
from tqdm import tqdm

In [2]:
# To get filenames in data folder
filelist = []
for file in os.listdir("./data/"):
    if file.endswith(".txt"):
        filelist.append(file)

In [3]:
def find_car_tour_mile(data):
    """A function calculates total millage for a taxi cab during roaming without passengers.
    Distance is calculated with distance package of geopy.
    
    Args:
        data (pd.DataFrame): The dataframe for each car
    
    Returns:
        car_total (float): total millage for a taxi
    """
    car_total = 0
    # divide data into "tours" according to "occupancy" changes
    data["tour"] = data.occupancy.diff().ne(0).cumsum()
    # What is "occupancy" of the first entrance
    if data["occupancy"].iloc[0]==0:
        start_index=1 
    else:
        start_index=2
    # Then each two steps is the "tour" with "occupancy=0" 
    for i in range(start_index,max(data["tour"])-1,2):
        subset_df = data[data["tour"] == i]
        latlongs = [(x, y) for x, y in zip(subset_df["latitude"],subset_df["longitude"])]
        sum_per_tour = 0
        # Calculate distance with two sequential geocodes
        for i in range(len(latlongs)-1):
            sum_per_tour += distance.distance(latlongs[i],latlongs[i+1]).miles
        car_total+= sum_per_tour
    return car_total

In [4]:
data = pd.DataFrame()
all_cars = {}
count = 0
# calculate monthly roaming millage from data files of each taxi cab
for i in tqdm(filelist):
    data_sub=pd.read_csv('data/'+i, sep=' ', header=None)
    car_name = i.split(".")[0].split("_")[-1]
    data_sub["car"] = car_name
    data_sub.columns = ["latitude", "longitude", "occupancy", "time","car"]
    data_sub["prev_occ"] = data_sub["occupancy"].shift(1).fillna(0).astype(int)
    data_sub["prev_time"] = data_sub["time"].shift(1).fillna(0).astype(int)
    data_sub["time_diff"] = data_sub["time"] - data_sub["prev_time"]
    # Remove data points on the sea
    is_on_land = globe.is_land(data_sub["latitude"], data_sub["longitude"])
    data_sub = data_sub[is_on_land]
    # Calculation
    car_miles = find_car_tour_mile(data_sub)
    all_cars[car_name] = car_miles


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 537/537 [17:09<00:00,  1.92s/it]


In [5]:
miles_data = pd.DataFrame.from_dict(all_cars, orient="index", columns=["miles"])
miles_data 

,miles
abboip,1939.659872
abcoij,78.763057
abdremlu,3829.357953
abgibo,2468.713641
abjoolaw,792.952103
...,...
uvburki,1230.349776
uvigcho,3370.064890
uvjeahot,2833.043103
uvjova,1622.033445


In [41]:
# Number of cars in fleet (combustine engine-powered)
fleet_comb_cars = len(all_cars)
# Predefined average passenger vehicle CO2 emission per mile (grams)
avg_co2_per_mile = 404
# Monthly CO2 emition per car (combustine engine-powered)
co2_g_month = sum(all_cars.values())/fleet_comb_cars*avg_co2_per_mile

In [42]:
sum(all_cars.values())/fleet_comb_cars

2279.0120504238494

In [37]:
# Taxi cab fleet is changing at the rate of 15% per month (according to total number of cars)
change_per_month = round(fleet_comb_cars*0.15)
pot_reduction = 0
for i in range(12):
    print("month: ",i)
    print("fleet combustion cars: ",fleet_comb_cars)
    # Gram to tonne conversion 
    gram_to_tonne = 1e-6
    # Cummulative potential reduction by month
    pot_reduction += fleet_comb_cars*co2_g_month*gram_to_tonne
    print('total potential due to roaming (tonne): {:.2f}'.format(pot_reduction))
    # Number of cars (combustine engine-powered) for the next month
    fleet_comb_cars = max(0, fleet_comb_cars-change_per_month)

month:  0
fleet combustion cars:  537
total potential due to roaming (tonne): 494.43
month:  1
fleet combustion cars:  456
total potential due to roaming (tonne): 914.28
month:  2
fleet combustion cars:  375
total potential due to roaming (tonne): 1259.55
month:  3
fleet combustion cars:  294
total potential due to roaming (tonne): 1530.24
month:  4
fleet combustion cars:  213
total potential due to roaming (tonne): 1726.35
month:  5
fleet combustion cars:  132
total potential due to roaming (tonne): 1847.89
month:  6
fleet combustion cars:  51
total potential due to roaming (tonne): 1894.84
month:  7
fleet combustion cars:  0
total potential due to roaming (tonne): 1894.84
month:  8
fleet combustion cars:  0
total potential due to roaming (tonne): 1894.84
month:  9
fleet combustion cars:  0
total potential due to roaming (tonne): 1894.84
month:  10
fleet combustion cars:  0
total potential due to roaming (tonne): 1894.84
month:  11
fleet combustion cars:  0
total potential due to roam

In [43]:
# Taxi cab fleet is changing at the rate of 15% per month (according to combustion engine-powered cars)
pot_reduction = 0
for i in range(12):
    print("month: ",i)
    print("fleet combustion cars: ",fleet_comb_cars)
    # Gram to tonne conversion 
    gram_to_tonne = 1e-6
    # Cummulative potential reduction by month
    pot_reduction += fleet_comb_cars*co2_g_month*gram_to_tonne
    print('total potential due to roaming (tonne): {:.2f}'.format(pot_reduction))
    # Number of cars (combustine engine-powered) for the next month
    fleet_comb_cars = round(fleet_comb_cars*0.85)

month:  0
fleet combustion cars:  537
total potential due to roaming (tonne): 494.43
month:  1
fleet combustion cars:  456
total potential due to roaming (tonne): 914.28
month:  2
fleet combustion cars:  388
total potential due to roaming (tonne): 1271.52
month:  3
fleet combustion cars:  330
total potential due to roaming (tonne): 1575.35
month:  4
fleet combustion cars:  280
total potential due to roaming (tonne): 1833.16
month:  5
fleet combustion cars:  238
total potential due to roaming (tonne): 2052.29
month:  6
fleet combustion cars:  202
total potential due to roaming (tonne): 2238.27
month:  7
fleet combustion cars:  172
total potential due to roaming (tonne): 2396.64
month:  8
fleet combustion cars:  146
total potential due to roaming (tonne): 2531.06
month:  9
fleet combustion cars:  124
total potential due to roaming (tonne): 2645.23
month:  10
fleet combustion cars:  105
total potential due to roaming (tonne): 2741.91
month:  11
fleet combustion cars:  89
total potential d